In [1]:
import SimFunctions
import SimClasses
import SimRNG_Modified
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import math
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from scipy.stats import probplot, kstest, t
from random import choice
import pickle
from copy import deepcopy

warnings.filterwarnings("ignore")
np.random.seed(1)
ZSimRNG = SimRNG_Modified.InitializeRNSeed()


# Data Loading & Preprocessing 

In [2]:
#subset_df = pd.read_csv("10_station_subset.csv")
subset_df = pd.read_csv("top20_station_subset.csv")
subset_df['End Station Id'] = subset_df['End Station Id'].astype(int)


## Arrival Rates

In [3]:
# Convert the start time and end time to minutes
subset_df['Start Time'] = pd.to_datetime(subset_df['Start Time'])
subset_df['End Time'] = pd.to_datetime(subset_df['End Time'])
subset_df['Start Time (per 30min)'] = (subset_df['Start Time'].dt.hour * 60 + (subset_df['Start Time'].dt.minute // 30) * 30 ) / 30
subset_df['End Time (per 30min)'] = (subset_df['End Time'].dt.hour * 60 + (subset_df['End Time'].dt.minute // 30) * 30 ) / 30

# Group the data by station and 30-minute interval, and count the number of trips that started in each group
Start_Station_HalfHour_Arrivals = subset_df.groupby([subset_df['Start Station Name'],subset_df['Start Station Id'], subset_df['Start Time (per 30min)']]).size().reset_index(name='ArrivalRate (per 30min)')
#Start_Station_HalfHour_Arrivals = subset_df.groupby([subset_df['Start Station Name'], subset_df['Start Station Id'], subset_df['End Station Name'], subset_df['End Station Id'], subset_df['Start Time (per 30min)']]).size().reset_index(name='ArrivalRate (per 30min)')

# Calculate the arrival rate at each station and 30-minute interval (trips per hour)
Start_Station_HalfHour_Arrivals['ArrivalRate (per min)'] = Start_Station_HalfHour_Arrivals['ArrivalRate (per 30min)'] / 30

# Display the result
arrival_df = Start_Station_HalfHour_Arrivals.sort_values(by="Start Time (per 30min)")


## Probability of Destinations

- After finding the probabilities of arriving to a destination from a specific stations, I filled any NA with 0.01 and then normalized to account for random arrivals to different destinations that was not in the data

In [4]:
subset_df['Start Time'] = pd.to_datetime(subset_df['Start Time'])
subset_df['End Time'] = pd.to_datetime(subset_df['End Time'])
subset_df['Start Time (per 30min)'] = (subset_df['Start Time'].dt.hour * 60 + (subset_df['Start Time'].dt.minute // 30) * 30 ) / 30
subset_df['End Time (per 30min)'] = (subset_df['End Time'].dt.hour * 60 + (subset_df['End Time'].dt.minute // 30) * 30 ) / 30

start_end_station_halfhour_trips = subset_df.groupby([subset_df['Start Station Name'], subset_df['Start Station Id'], subset_df['End Station Name'], subset_df['End Station Id'], subset_df['Start Time (per 30min)']]).size().reset_index(name='NumOfTrips')
total_trips = start_end_station_halfhour_trips.groupby(['Start Station Name', 'Start Time (per 30min)'])['NumOfTrips'].sum().reset_index(name='TotalTrips')

start_end_station_prob = pd.merge(start_end_station_halfhour_trips, total_trips, on=['Start Station Name', 'Start Time (per 30min)'])
start_end_station_prob['Probability'] = start_end_station_prob['NumOfTrips'] / start_end_station_prob['TotalTrips']

station_vs_Dest_vs_halfhour = start_end_station_prob.pivot(index=['Start Station Name', 'Start Time (per 30min)'], columns=['End Station Name'], values='Probability').fillna(0)
name_prob_df = station_vs_Dest_vs_halfhour.sort_values(by="Start Time (per 30min)")

In [5]:
station_vs_Dest_vs_halfhour = start_end_station_prob.pivot(index=['Start Station Id', 'Start Time (per 30min)'], columns=['End Station Id'], values='Probability').fillna(0)
prob_df = station_vs_Dest_vs_halfhour.sort_values(by="Start Time (per 30min)")



## Trip Durations

### Different Stations Destinations

In [6]:
with open('top20_diff_google_bike_trip_est.pickle', 'rb') as f:
    google_bike_trip = pickle.load(f)

diff_stations_subset_df = pd.read_csv("diff_stations_subset_df.csv")
google = pd.DataFrame((np.array(google_bike_trip)), columns=["Google"])
observed = pd.DataFrame(np.array((diff_stations_subset_df["Trip_Duration"].values)), columns=["Observed"])
trip_reg_df = pd.DataFrame({"Observed": np.array((diff_stations_subset_df["Trip_Duration"].values)*60), "Google": np.array(google_bike_trip)})

In [7]:
np.random.seed(1)

X = np.log(trip_reg_df["Google"].values)
y = np.log(trip_reg_df["Observed"].values)

X = np.array(X).reshape(-1, 1)

# set regression through the origin
model = LinearRegression(fit_intercept=True)
model.fit(X, y)
predictions = model.predict(X)
score = model.score(X, y)
beta = model.coef_[0]
intercept = model.intercept_

residuals = y - predictions
residual_var = np.var(residuals)
residual_mean = np.mean(residuals)

print("Error Mean", residual_mean)
print("Residual Variance:", residual_var)
print()
print('Beta:', beta)
print('Intercept:', intercept)
print("R^2:", score)


Error Mean -1.5261626468615904e-16
Residual Variance: 0.25580459588191384

Beta: 0.4513542745881248
Intercept: 3.974270551809947
R^2: 0.3143043103068821


In [8]:
np.random.seed(1)

X = np.log(trip_reg_df["Google"].values)
y = np.log(trip_reg_df["Observed"].values)

X_i = np.array(X).reshape(-1, 1)

# set regression through the origin
model = LinearRegression(fit_intercept=True)
model.fit(X_i, y)
predictions = model.predict(X_i)
score = model.score(X_i, y)
beta = model.coef_[0]
intercept = model.intercept_

residuals = y - predictions
residual_mean = np.mean(residuals)
residual_var = np.var(residuals)
residual_std = np.std(residuals)

error_sd = np.sqrt(residual_var)
errors = np.random.normal(loc=residual_mean, scale=residual_std, size=len(residuals))
epsilon = residuals - errors

new_X = X + epsilon
new_X_i = np.array(new_X).reshape(-1, 1)

# Fit linear regression model
model = LinearRegression(fit_intercept=True)
model.fit(new_X_i, y)
new_predictions = model.predict(new_X_i)

# Calculate R-squared and print results
r2 = model.score(new_X_i, y)
new_beta = model.coef_[0]
new_intercept = model.intercept_

new_residuals = y - new_predictions
new_residual_mean = np.mean(new_residuals)
new_residual_var = np.var(new_residuals)
new_residual_std = np.std(new_residuals)
print("Pre Residual Mean", residual_mean)
print("Pre Residual Variance:", residual_var)
print()

print("New Resiudal Mean", new_residual_mean)
print("New Residual Variance:", new_residual_var)
print()
print('Beta:', new_beta)
print('Intercept:', new_intercept)
print("R^2:", r2)

# plt.scatter(new_X_i, y, color='darkblue', alpha=0.1)
# plt.plot(new_X_i, new_predictions, color='red')
# plt.title("(EPSILON) Linear Regression of ln(Google trips) vs ln(Observed Data)")
# plt.ylabel("ln(Observed Data)")
# plt.xlabel("ln(Google Trips)")
# plt.legend(["Google", "Real"], loc="lower right")
# plt.show()


Pre Residual Mean -1.5261626468615904e-16
Pre Residual Variance: 0.25580459588191384

New Resiudal Mean -5.530146832449613e-16
New Residual Variance: 0.12858666740184382

Beta: 0.47504569391691565
Intercept: 3.8372405086866
R^2: 0.6553176721259963


In [9]:
global E_x, SD_X

u = residual_mean
std = np.sqrt(residual_var)
E_x = np.exp(u + ((std**2)/2))
SD_x = np.exp(u + ((std**2)/2)) * np.sqrt(np.exp(std**2) - 1)
print(E_x, SD_x)


1.1364419645529011 0.6135737822915353


### Same Start and End Destination

In [10]:
same_stations_subset_df = pd.read_csv("same_stations_subset_df.csv")

### Average Trip Durations between Specific Stations

In [11]:
# Convert the start time and end time to minutes
subset_df['Start Time'] = pd.to_datetime(subset_df['Start Time'])
subset_df['End Time'] = pd.to_datetime(subset_df['End Time'])
subset_df['Start Time (per 30min)'] = (subset_df['Start Time'].dt.hour * 60 + (subset_df['Start Time'].dt.minute // 30) * 30 ) / 30
subset_df['End Time (per 30min)'] = (subset_df['End Time'].dt.hour * 60 + (subset_df['End Time'].dt.minute // 30) * 30 ) / 30

# Group the data by start and end station and 30-minute interval, and calculate the average trip duration in seconds for each group
Station_HalfHour_AvgDuration = subset_df.groupby([subset_df['Start Station Name'], subset_df['Start Station Id'], subset_df['End Station Name'], subset_df['End Station Id'], subset_df['Start Time (per 30min)']])['Trip_Duration'].mean().reset_index(name='Avg_Trip_Duration')

Station_HalfHour_AvgDuration['Avg_Trip_Duration'] = Station_HalfHour_AvgDuration['Avg_Trip_Duration']

avg_trip_duration = Station_HalfHour_AvgDuration.sort_values(by="Start Time (per 30min)")


# Classes & Functions

## Classes

- Decide on the time units
    - Min 

In [35]:
class Station:
    def __init__(self, station_id, level, capacity):
        self.id = station_id
        self.level = level
        self.capacity = capacity
        self.bikes = {}
        self.bike_list = []
        for i in range(level):
            bike_id = f"{station_id}-{i+1}"  # create unique bike ID
            self.bikes[bike_id] = True  # mark bike as available
            self.bike_list.append(bike_id)

    def rent_bike(self):
        # Request a bike from the station
        if self.level > 0:
            if self.bike_list:
                random_index = np.random.randint(0, len(self.bike_list))
                bike_id = self.bike_list.pop(random_index)
                self.level -= 1
                return bike_id
        return None

    def return_bike(self, bike_id):
        # # Return a bike to the station
        if self.level < self.capacity:
            self.bike_list.append(bike_id)
            self.level += 1

    def Get_Bike_List(self):
        return self.bike_list


class Customer:
    def __init__(self, customer_id, start_s_id, bike=None):
        self.customer_id = customer_id
        self.start_s_id = start_s_id
        self.end_s_id = 0
        self.station_level = 0
        self.bike = bike
        self.T = 0
        self.time = 0
        self.Min = None
        self.Trip_Time = 0

    def rent_bike(self):
        station = StationDict[self.start_s_id]
        #print(f"   Customer Arrives at S{station.id} with Level: {station.level}")

        if station.level > 0:
            self.bike = station.rent_bike()
            self.station_level = station.level
            #print(f"    [Customer Rent Bikes] Customer ID: {self.customer_id} | Bike ID {self.bike} || Start Time: {self.T}:{self.Min} || From: S{self.start_s_id} -> Remaining Level: {self.station_level}")
            self.Departure()
        #else:
            #print(f"    -   (EMPTY) -- Customer {self.customer_id} CANNOT RENT BIKE -- EMPTY STATION {self.start_s_id} w/ level {self.station_level} -- (EMPTY)")

    def return_bike(self, end_station, bike):
        destination_station = StationDict[end_station]
        destination_station.return_bike(bike)
        self.station_level = destination_station.level
    
    def Departure(self):
        self.end_s_id = int(self.Destination())
        end_station = StationDict[self.end_s_id]
        
        trip_time = self.TripDuration()
        self.Trip_Time = trip_time
        
        # print(f"        [Customer Rents Bike and Departs]: Customer ID: {self.customer_id} | Bike ID: {self.bike}")
        # print(f"            - Start Time:{self.time}:{self.Min}")
        # print(f"            - From: S{self.start_s_id} -> Level {self.station_level} | To: S{end_station.id} -> Level {end_station.level}")
        # print(f"            - Expected Trip Time: {self.Trip_Time} min")
        
        SimFunctions.Schedule(Calendar, "Bike_Arrival", trip_time)
        return self.end_s_id
    

######
#HELPER FUNCTIONS
######
    def Destination(self):
        end_s_id = None
        while end_s_id is None:
            end_s_id = self.ChoosingRoute(prob_df, start_s_id=self.start_s_id, start_time=self.T)
        return end_s_id


    def ChoosingRoute(self, prob_df, start_s_id, start_time):
        while True:
            try:
                start_row = prob_df.loc[(start_s_id, max(0, start_time))]
                probs = start_row.values
                end_s_id = np.random.choice(start_row.index, p=probs)
                condition = False
                return end_s_id

            except KeyError:
                print(
                    f"Choosing Route Error NO DATA found for start station: '{start_s_id}' and start T: '{start_time}'")

                start_time += 1
                if start_s_id == 7217 and start_time == 19:
                    start_time = 20

    def TripDuration(self):
        
        if self.start_s_id == self.end_s_id: #sample from empherical df if same start and end station
            time_df = subset_df[subset_df["Start Time (per 30min)"] == self.T]
            same_station_subset = time_df.loc[time_df['Start Station Id'] == time_df['End Station Id']]['Trip_Duration'].values
            duration_data = np.random.choice(same_station_subset)
            
        else:
            T = self.T
            condition = True
            while condition:
                try:
                    duration_data = avg_trip_duration[(avg_trip_duration["Start Time (per 30min)"] == T) &
                                            (avg_trip_duration["Start Station Id"] == self.start_s_id) &
                                            (avg_trip_duration["End Station Id"] == self.end_s_id)]["Avg_Trip_Duration"].values[0]
                    condition = False
                except KeyError:
                    print(f"Trip Duration Error NO DATA found for start station: '{self.start_s_id}' and start T: '{T}'")
                    T -= 1
                    if T < 0:  
                        return 2
                except IndexError:
                    return 2
        trip_time = duration_data * SimRNG_Modified.Lognormal(ZSimRNG, E_x, SD_x**2, 4)
        trip_time = min(35, np.round(trip_time))
        trip_time = max(2, trip_time)
        return trip_time


## Functions

In [13]:
def Start():
    SimFunctions.Schedule(Calendar, "Customer_Arrival",
                          SimRNG_Modified.Expon(ZSimRNG, 0, 1))

def NextCustomerID():
    if not hasattr(NextCustomerID, "counter"):
        NextCustomerID.counter = 0
    NextCustomerID.counter += 1
    return NextCustomerID.counter


def CI_95(data):
    a = np.array(data)
    n = len(a)
    m = np.mean(a)
    sd = np.std(a, ddof=1)
    hw = 1.96*sd / np.sqrt(n)
    return m, "+/-", hw


def inital_Customer_Arrival_Rate(T):
    temp_df = arrival_df[arrival_df["Start Time (per 30min)"] == T]
    arrival_rates = temp_df["ArrivalRate (per min)"].values
    possible_station_ids = temp_df["Start Station Id"].values
    arrival_rates = arrival_df[(arrival_df["Start Time (per 30min)"] == T)]["ArrivalRate (per min)"].values
    return arrival_rates[:5], possible_station_ids[:5]


def inital_Customer_Arrival(empty_error, CustomerList, T, minute):
    arrival_rates, multi_station_id = inital_Customer_Arrival_Rate(T)
    for i, station_id in enumerate(multi_station_id):
        arrival_rate = arrival_rates[i]
        station = StationDict[station_id]
        customer_id = NextCustomerID()
        customer = Customer(customer_id, station_id)
        customer.start_s_id = station_id
        customer.station_level = station.level
        customer.T = T
        customer.time = T//2
        customer.Min = minute

        mu = 1/arrival_rate
        inter_arrival_time = np.round(SimRNG_Modified.Expon(ZSimRNG, mu, 1))
        SimFunctions.Schedule(Calendar, "Customer_Arrival",
                            max(2, inter_arrival_time))
        # Store values in global lists
        start_time_list.append(T//2)
        inter_arrival_time_list.append(inter_arrival_time)
        arrival_time_list.append(arrival_rate)
        start_station_id_list.append(station_id)

        ################################################################
        # STATION EMPTY
        ################################################################
        if customer.station_level == 0:
            #print(f"    (EMPTY) -- Customer {customer.customer_id} CANNOT RENT BIKE | S{customer.start_s_id} -> level {customer.station_level} -- (EMPTY)")
            empty_error += 1
        else:
            CustomerList.append(customer)
            customer.rent_bike()
    
    return empty_error


def Customer_Arrival_Rate(T):
    temp_df = arrival_df[arrival_df["Start Time (per 30min)"] == T]
    arrival_rates = temp_df["ArrivalRate (per min)"].values
    possible_station_ids = temp_df["Start Station Id"].values
    arrival_rates = arrival_df[(
        arrival_df["Start Time (per 30min)"] == T)]["ArrivalRate (per min)"].values
    selected_station_id = np.random.choice(
        possible_station_ids, p=(arrival_rates / arrival_rates.sum()))
    arrival_rate = arrival_df[(arrival_df["Start Time (per 30min)"] == T) & (
        arrival_df["Start Station Id"] == selected_station_id)]["ArrivalRate (per min)"].values[0]
    return arrival_rate, selected_station_id


def Customer_Arrival(empty_error, CustomerList, T, minute):
    global inter_arrival_time_list, arrival_time_list, start_station_id_list, start_time_list
    arrival_rate, station_id = Customer_Arrival_Rate(T)
    station = StationDict[station_id]
    customer_id = NextCustomerID()
    customer = Customer(customer_id, station_id)
    customer.start_s_id = station_id
    customer.station_level = station.level
    customer.T = T
    customer.time = T//2
    customer.Min = minute

    mu = 1/arrival_rate
    inter_arrival_time = np.round(SimRNG_Modified.Expon(ZSimRNG, mu, 1))
    SimFunctions.Schedule(Calendar, "Customer_Arrival",
                          max(2, inter_arrival_time))
    # Store values in global lists
    start_time_list.append(T)
    inter_arrival_time_list.append(inter_arrival_time)
    arrival_time_list.append(arrival_rate)
    start_station_id_list.append(station_id)

################################################################
# STATION EMPTY
################################################################
    if customer.station_level == 0:
        #print(f"    (EMPTY) -- Customer {customer.customer_id} CANNOT RENT BIKE | S{customer.start_s_id} -> level {customer.station_level} -- (EMPTY)")
        empty_error += 1
    else:
        CustomerList.append(customer)
        customer.rent_bike()

    return empty_error


def Bike_Arrival(Full_Error, CustomerList, T, minute):
    global end_station_id_list, end_time_list, trip_time_list

    for customer in CustomerList:
        end_s_id = customer.end_s_id
        end_station = StationDict[end_s_id]
        if end_s_id != 0:
            if end_station.level < end_station.capacity:
                end_time_minutes = (T//2) * 60 + minute
                start_time_minutes = customer.time * 60 + customer.Min
                total_trip_time = end_time_minutes - start_time_minutes
                # if customer.end_s_id == end_station.id and customer.bike is not None and total_trip_time >= customer.Trip_Time:
                if customer.end_s_id == end_station.id and total_trip_time >= customer.Trip_Time:
                    customer.return_bike(customer.end_s_id, customer.bike)
                    # print(f"                [BIKE RETURNED] Customer ID: {customer.customer_id} | Bike ID: {customer.bike}")
                    # print(f"                    - Start Time:{customer.time}:{customer.Min} - End Time:{T//2}:{minute}")
                    # print(f"                    - Expected Trip Time: {customer.Trip_Time} min")
                    # print(f"                    - Total Trip Time: {total_trip_time} min")
                    # print(f"                    - From: S{customer.start_s_id} | To: S{end_station.id} -> Level {end_station.level}")
                    end_station_id_list.append(end_station.id)
                    end_time_list.append(T)
                    trip_time_list.append(total_trip_time)
                    CustomerList.remove(customer)

    ################################################################
    # STATION FULL
    ################################################################
            else:
                for customer in CustomerList:
                    customer_end_station = StationDict[customer.start_s_id]
                    end_time_minutes = (T//2) * 60 + minute
                    start_time_minutes = customer.time * 60 + customer.Min
                    total_trip_time = end_time_minutes - start_time_minutes
                    if customer.end_s_id == customer_end_station.id and customer_end_station.level >= customer_end_station.capacity and total_trip_time >= customer.Trip_Time:
                        temp_customer = customer
                        end_id = end_station.id
                        #print(f"     (FULL) -- Start Time:{customer.time}:{customer.Min} - End Time:{T//2}:{minute} || Customer ID: {customer.customer_id} || To: S{end_id} -> Level {end_station.level} | Capacity {end_station.capacity} || From: S{customer.start_s_id} -- (FULL)")
                        Full_Error += 1
                        Retrial(temp_customer=temp_customer,
                                end_id=end_id, T=T, minute=minute)
                        return Full_Error
    return Full_Error


def Retrial(temp_customer, end_id, T, minute):
    customer = temp_customer
    customer.start_s_id = end_id
    customer.T = T
    customer.time = T//2
    customer.Min = minute
    customer.end_s_id = customer.Destination()
    #print(f"    [TRAVELS TO NEW STATION] Start Time:{customer.time}:{customer.Min} || Customer ID: {customer.customer_id} travels to S{customer.end_s_id}")
    trip_time = customer.TripDuration()
    SimFunctions.Schedule(Calendar, "Bike_Arrival", trip_time)


# Simulation

In [14]:
def CI_95(data):
    a = np.array(data)
    n = len(a)
    m = np.mean(a)
    sd = np.std(a, ddof=1)
    hw = 1.96*sd / np.sqrt(n)
    return m, "+/-", hw


In [15]:
CI_total_error_list = []
total_error_list = []

for i in range(3):
    NextCustomerID.counter = 0

    ZSimRNG = SimRNG_Modified.InitializeRNSeed()

    Calendar = SimClasses.EventCalendar()
    TheCTStats = []
    TheDTStats = []
    TheQueues = []
    TheResources = []
    Stations = []
    CustomerList = []
    CI_Full_Error_list = []
    CI_Empty_Error_list = []

    CI_inter_arrival_time_list = []
    CI_arrival_time_list = []
    CI_start_station_id_list = []
    CI_end_station_id_list = []
    CI_start_time_list = []
    CI_end_time_list = []
    CI_trip_time_list = []




    for days in range(0, 1, 1):
        Full_Error = 0
        Empty_Error = 0
        inital_count = 0

        inter_arrival_time_list = []
        arrival_time_list = []
        start_station_id_list = []
        end_station_id_list = []
        start_time_list = []
        end_time_list = []
        trip_time_list = []
    ############################################################################################################################################################################
        # Initialize the stations and create a dictionary mapping station IDs to Station instances
        unique_stations = np.unique(subset_df["Start Station Id"].values)
        num_stations = len(unique_stations)
        total_capacity = 500
        total_bikes = 399
        
        Stations = {}
        level_sum = 0
        level_sum = 0
        capacity_sum = 0
        
        capacity_per_station = total_capacity // num_stations

        for i, station in enumerate(unique_stations):
            x_i = total_bikes //19
            r_i = capacity_per_station
            if i == num_stations - 1:
                # Allocate the remaining capacity to the last station
                r_i = total_capacity - capacity_per_station * (num_stations - 1)
            Stations[station] = Station(station_id=station, level=x_i, capacity=r_i)
            level_sum += x_i
            for station_id, station in Stations.items():
                capacity_sum += station.capacity


        StationDict = {station.id: station for station in Stations.values()}

        # Print the initial bike list for each station
        val_level_sum = []
        val_capacity_sum = []
        count = 0
        for station_id, station in Stations.items():
            count += 1
            #print(f"Station {station_id} Initial Bike List: {station.Get_Bike_List()}")
            val_level_sum.append(station.level)
            val_capacity_sum.append(station.capacity)
        #print("Number of Stations", count)
        #print("Level Sum", sum(val_level_sum))
        #print("Capacity Sum", sum(val_capacity_sum))

    ############################################################################################################################################################################
        
        SimFunctions.SimFunctionsInit(Calendar, TheQueues, TheCTStats, TheDTStats, TheResources)
        SimFunctions.Schedule(Calendar, "Start", 0)

        NextEvent = Calendar.Remove()
        SimClasses.Clock = NextEvent.EventTime
        if NextEvent.EventType == "Start":
            Start()

    ############################################################################################################################################################################
    # SIMULATION RUN
        for T in range(15, 24):  # T = hours intervals
            inital_count += 1
            hour = T // 2
            minute = 00 if T % 2 == 0 else 30
            unit = 'PM' if hour >= 12 else 'AM'
            #print()
            #print("Interval:", T)
            mini = 0
            SimFunctions.Schedule(Calendar, "inital_Customer_Arrival", 0)

            if inital_count == 2:
                Full_Error = 0
                Empty_Error = 0

            while True:
                #print("Clock: {:02d}:{:02d} {:s}".format(hour, minute, unit))
                NextEvent = Calendar.Remove()
                SimClasses.Clock = NextEvent.EventTime
                minute = int((SimClasses.Clock) % 60)
                if SimClasses.Clock >= (T+1) * 30:
                    break
                if NextEvent.EventType == "inital_Customer_Arrival":
                    Empty_Error = inital_Customer_Arrival(
                        Empty_Error, CustomerList, T, minute)
                elif NextEvent.EventType == "Customer_Arrival":
                    Empty_Error = Customer_Arrival(
                        Empty_Error, CustomerList, T, minute)
                elif NextEvent.EventType == "Bike_Arrival":
                    Full_Error = Bike_Arrival(Full_Error, CustomerList, T, minute)

    ##################################################################################################################################################
    # OPTIMIZE
        objective_fun = Full_Error + Empty_Error
        print(objective_fun)
        total_error_list.append(objective_fun)

    # OPTIMZE

    CI_Full_Error_list.append(Full_Error)
    CI_Empty_Error_list.append(Empty_Error)
    CI_total_error_list.append(CI_95(total_error_list))

    CI_inter_arrival_time_list.append(inter_arrival_time_list)
    CI_arrival_time_list.append(arrival_time_list)
    CI_start_station_id_list.append(start_station_id_list)
    CI_end_station_id_list.append(end_station_id_list)
    CI_start_time_list.append(start_time_list)
    CI_end_time_list.append(end_time_list)
    CI_trip_time_list.append(trip_time_list)
        
    print(f"End of Day {days}")
    print("--------------------------------------------------------------------------------------------------------------------------------------------------")
    print()

# Errors_DF = pd.DataFrame({"Full Error": CI_Full_Error_list,
#                         "Empty Error": CI_Empty_Error_list,
#                         "Total Error": CI_total_error_list})

BikeSim_DF = pd.DataFrame({"Start Station ID": CI_start_station_id_list,
                        "End Station ID": CI_end_station_id_list,
                        "Arrival Rate": CI_arrival_time_list,
                        "Interarrival Rate": CI_inter_arrival_time_list,
                        "Start Time": CI_start_time_list,
                        "End Time": CI_end_time_list,
                        "Trip Time": CI_trip_time_list})

print(f"Num of Full Errors: {CI_Full_Error_list}")
print(f"Num of Empty Errors: {CI_Empty_Error_list}")
print(f"Total Errors: {CI_total_error_list}")


463
End of Day 0
--------------------------------------------------------------------------------------------------------------------------------------------------

461
End of Day 0
--------------------------------------------------------------------------------------------------------------------------------------------------

Choosing Route Error NO DATA found for start station: '7217' and start T: '19'
398
End of Day 0
--------------------------------------------------------------------------------------------------------------------------------------------------

Num of Full Errors: [45]
Num of Empty Errors: [353]
Total Errors: [(463.0, '+/-', nan), (462.0, '+/-', 1.9599999999999997), (440.6666666666667, '+/-', 41.82864303055715)]


In [16]:
CI_95(total_error_list)


(440.6666666666667, '+/-', 41.82864303055715)

# Flow Rate Calculation

## Total Errors

In [17]:
# print(f"Num of Full Errors: {CI_Full_Error_list}")
# print(f"Num of Empty Errors: {CI_Empty_Error_list}")
# print(f"Total Errors: {total_error_list}")


# Flow Rates

In [18]:
i = 0
station_id = BikeSim_DF["Start Station ID"].values[i]
end_station_id = BikeSim_DF["End Station ID"].values[i]
start_times = BikeSim_DF["Start Time"].values[i]
end_times = BikeSim_DF["End Time"].values[i]
trip_times = BikeSim_DF["Trip Time"].values[i]
Arrival_Rates = BikeSim_DF["Arrival Rate"].values[i]
Interarrival_Rates = BikeSim_DF["Interarrival Rate"].values[i]

print("List Lengths")
print(f"Start ID: {len(station_id)} | End ID: {len(end_station_id)}")
print(f"Start Time {len(start_times)} | End Time {len(end_times)}")
print(f"Trip Time {len(trip_times)}")
print(f"Arrival Rate {len(Arrival_Rates)} | Interarrival Rate {len(Interarrival_Rates)}")

start_flow_df = pd.DataFrame({"Start ID": station_id, "Start Time": start_times})
end_flow_df = pd.DataFrame({"End ID": end_station_id,"End Time": end_times, "Trip Time": trip_times})
arrival_flow_df = pd.DataFrame({"Start ID": station_id, "Arrival Rates": Arrival_Rates})
interarrival_flow_df = pd.DataFrame({"Start ID": station_id, "Arrival Rates": Interarrival_Rates})

start_flow_count = start_flow_df.pivot_table(index='Start ID', columns='Start Time', aggfunc='size', fill_value=0)
start_flow_count = start_flow_count.loc[:, 16:] 
end_flow_count = end_flow_df.pivot_table(index='End ID', columns='End Time', aggfunc='size', fill_value=0)
end_flow_count = end_flow_count.loc[:, 16:]
flow_rate = end_flow_count - start_flow_count


List Lengths
Start ID: 3027 | End ID: 2249
Start Time 3027 | End Time 2249
Trip Time 2249
Arrival Rate 3027 | Interarrival Rate 3027


## Trip Time Verfication

In [19]:
# fig, ax = plt.subplots(1, 2, figsize=(16, 6))

# # Plot for T=16
# T = 20
# real_Trip_Duration = subset_df[subset_df["End Time (per 30min)"] == T][["End Time (per 30min)", "End Station Id", "Trip_Duration"]].sort_values(by="End Time (per 30min)", ascending=False)
# avg_trip_time = np.round(real_Trip_Duration.groupby("End Station Id")["Trip_Duration"].mean())

# sample_trip = end_flow_df[end_flow_df["End Time"]== T]["End ID"].value_counts()
# sample_trips = pd.DataFrame({"End Station Id": sample_trip.index, "Sample Trip Times": sample_trip.values})

# comparison_df = pd.merge(avg_trip_time, sample_trips,
#                          on="End Station Id", how="outer")
# comparison_df.columns = ["End Station Id","Real Trip Times", "Sample Trip Times"]
# comparison_df = comparison_df.sort_values(by="End Station Id")

# bar_width = 0.35
# ax[0].barh(np.arange(len(comparison_df)), comparison_df["Real Trip Times"],
#            color="darkblue", alpha=0.5, height=bar_width, label="Real Trip Time")
# ax[0].barh(np.arange(len(comparison_df))+bar_width, comparison_df["Sample Trip Times"],
#            color="red", alpha=0.5, height=bar_width, label="Sample Trip Time")
# ax[0].set_yticks(np.arange(len(comparison_df))+bar_width / 2)
# ax[0].set_yticklabels(comparison_df["End Station Id"])
# ax[0].set_ylabel("End Station ID")
# ax[0].set_xlabel("Customer Trip Time")
# ax[0].set_title("Trip Durations at 9:00 AM")
# ax[0].legend(loc='upper right', fontsize='small')

# # Plot for T=17
# T = 21
# real_Trip_Duration = subset_df[subset_df["End Time (per 30min)"] == T][["End Time (per 30min)", "End Station Id", "Trip_Duration"]].sort_values(by="End Time (per 30min)", ascending=False)
# avg_trip_time = np.round(real_Trip_Duration.groupby("End Station Id")["Trip_Duration"].mean())


# sample_trip = end_flow_df[end_flow_df["End Time"]
#                           == T]["End ID"].value_counts()
# sample_trips = pd.DataFrame(
#     {"End Station Id": sample_trip.index, "Sample Trip Times": sample_trip.values})

# comparison_df = pd.merge(avg_trip_time, sample_trips,
#                          on="End Station Id", how="outer")
# comparison_df.columns = ["End Station Id",
#                          "Real Trip Times", "Sample Trip Times"]
# comparison_df = comparison_df.sort_values(by="End Station Id")

# ax[1].barh(np.arange(len(comparison_df)), comparison_df["Real Trip Times"],color="darkblue", alpha=0.5, height=bar_width, label="Real Trip Time")
# ax[1].barh(np.arange(len(comparison_df))+bar_width, comparison_df["Sample Trip Times"], color="red", alpha=0.5, height=bar_width, label="Sample Trip Time")
# ax[1].set_yticks(np.arange(len(comparison_df))+bar_width / 2)
# ax[1].set_yticklabels(comparison_df["End Station Id"])
# ax[1].set_ylabel("End Station ID")
# ax[1].set_xlabel("Customer Trip Time")
# ax[1].set_title("Trip Durations at 9:30 AM")
# ax[1].legend(loc='upper right', fontsize='small')
# fig.suptitle("Comparison of Real and Sample Trip Durations")
# fig.tight_layout()
# plt.show()


## Arrival Rate Verfication

In [20]:
# fig, axs = plt.subplots(1, 2, figsize=(16, 6))

# # First subplot for T=16
# T = 18
# real_8am_S_ID = arrival_df[arrival_df["Start Time (per 30min)"] == T][["Start Station Id", "ArrivalRate (per 30min)"]].sort_values(by="ArrivalRate (per 30min)", ascending=False)
# sample_8am_s_ID = start_flow_df[start_flow_df["Start Time"]== T]["Start ID"].value_counts()
# sample_8am_s_ID_df = pd.DataFrame({"Start Station Id": sample_8am_s_ID.index, "ArrivalRate (per 30min)": sample_8am_s_ID.values})

# comparison_df = pd.merge(real_8am_S_ID, sample_8am_s_ID_df, on="Start Station Id", how="outer")
# comparison_df.columns = ["Start Station Id", "Real ArrivalRate (per 30min)", "Sample ArrivalRate (per 30min)"]
# comparison_df = comparison_df.sort_values(by="Start Station Id")

# bar_width = 0.35
# axs[0].barh(np.arange(len(comparison_df)), comparison_df["Real ArrivalRate (per 30min)"],color="blue", alpha=0.5, height=bar_width, label="Real Arrival Rate")
# axs[0].barh(np.arange(len(comparison_df))+bar_width, comparison_df["Sample ArrivalRate (per 30min)"],color="orange", alpha=0.7, height=bar_width, label="Sample Arrival Rate")
# axs[0].set_yticks(np.arange(len(comparison_df))+bar_width / 2)
# axs[0].set_yticklabels(comparison_df["Start Station Id"])
# axs[0].set_ylabel("Start Station ID")
# axs[0].set_xlabel("Customer Arrival Rate (per 30 min)")
# axs[0].set_title("Arrival Rate at 9 AM")
# axs[0].legend(loc='upper right', fontsize='small')

# # Second subplot for T=17
# T = 19
# real_8am_S_ID = arrival_df[arrival_df["Start Time (per 30min)"] == T][["Start Station Id", "ArrivalRate (per 30min)"]].sort_values(by="ArrivalRate (per 30min)", ascending=False)
# sample_8am_s_ID = start_flow_df[start_flow_df["Start Time"]== T]["Start ID"].value_counts()
# sample_8am_s_ID_df = pd.DataFrame({"Start Station Id": sample_8am_s_ID.index, "ArrivalRate (per 30min)": sample_8am_s_ID.values})

# comparison_df = pd.merge(real_8am_S_ID, sample_8am_s_ID_df,on="Start Station Id", how="outer")
# comparison_df.columns = ["Start Station Id", "Real ArrivalRate (per 30min)", "Sample ArrivalRate (per 30min)"]
# comparison_df = comparison_df.sort_values(by="Start Station Id")

# bar_width = 0.35
# axs[1].barh(np.arange(len(comparison_df)), comparison_df["Real ArrivalRate (per 30min)"],color="blue", alpha=0.5, height=bar_width, label="Real Arrival Rate")
# axs[1].barh(np.arange(len(comparison_df))+bar_width, comparison_df["Sample ArrivalRate (per 30min)"],color="orange", alpha=0.7, height=bar_width, label="Sample Arrival Rate")
# axs[1].set_yticks(np.arange(len(comparison_df))+bar_width / 2)
# axs[1].set_yticklabels(comparison_df["Start Station Id"])
# axs[1].set_ylabel("Start Station ID")
# axs[1].set_xlabel("Customer Arrival Rate (per 30 min)")
# axs[1].set_title("Arrival Rate at 9:30 AM")
# axs[1].legend(loc='upper right', fontsize='small')

# fig.suptitle("Comparison of Real and Sample Arrival Rates")
# fig.tight_layout()
# plt.show()


# Starting Solutions

## Equal Allocation

In [21]:
# unique_stations = np.unique(subset_df["Start Station Id"].values)
# num_stations = len(unique_stations)
# total_capacity = 500
# total_bikes = 399
# Stations = {}
# level_sum = 0
# level_sum = 0
# capacity_sum = 0


# for i, station in enumerate(unique_stations):
#     x_i = 399//19
#     r_i = capacity_per_station
#     if i == num_stations - 1:
#         # Allocate the remaining capacity to the last station
#         r_i = total_capacity - capacity_per_station * (num_stations - 1)
#     Stations[station] = Station(station_id=station, level=x_i, capacity=r_i)
#     level_sum += x_i
#     for station_id, station in Stations.items():
#         capacity_sum += station.capacity


# StationDict = {station.id: station for station in Stations.values()}

# # Print the initial bike list for each station
# val_level_sum = []
# val_capacity_sum = []
# count = 0
# for station_id, station in Stations.items():
#     count += 1
#     print(f"Station {station_id} Initial Bike List: {station.Get_Bike_List()}")
#     val_level_sum.append(station.level)
#     val_capacity_sum.append(station.capacity)
# print(count)

# print("Level Sum", sum(val_level_sum))
# print("Capacity Sum", sum(val_capacity_sum))


## Flow Rate Model Solution

In [22]:
# plt.figure(figsize=(8, 5))
# unique_stations = np.unique(subset_df["Start Station Id"].values)

# for station_ids in unique_stations[:3]:
#     start_flow = start_flow_count.loc[station_ids]
#     end_flow = end_flow_count.loc[station_ids]
#     flow_rate = end_flow - start_flow 
#     plt.plot(flow_rate.index, flow_rate.values, label=f'Station {station_ids}')

# plt.xlabel('Time')
# plt.ylabel('Flow Rate')
# plt.title('Flow Rate of Stations')
# plt.legend(loc='lower left', fontsize='small')
# plt.show()


In [23]:
# # Parameters
# num_stations = 20
# total_capacity = 500
# total_bikes = 399
# min_station_capacity = 12
# max_station_capacity = 32

# # Assuming start_flow_df and end_flow_df are available
# start_time = 16
# end_time = 25

# # Calculate flow rate using pandas pivot tables
# start_flow_count = start_flow_df.pivot_table(
#     index='Start ID', columns='Start Time', aggfunc='size', fill_value=0)
# start_flow_count = start_flow_count.loc[:, start_time:end_time]
# end_flow_count = end_flow_df.pivot_table(
#     index='End ID', columns='End Time', aggfunc='size', fill_value=0)
# end_flow_count = end_flow_count.loc[:, start_time:end_time]
# flow_rate = end_flow_count - start_flow_count

# # Calculate flow rate data as a dictionary
# flow_rate_data = flow_rate.sum(axis=1).to_dict()

# # Calculate capacities and levels based on flow rates
# station_ids = list(flow_rate_data.keys())
# flow_rates = np.array(list(flow_rate_data.values()))

# # Initialize station capacities
# station_capacities = np.clip(flow_rates + total_capacity //
#                              num_stations, min_station_capacity, max_station_capacity)

# # Initialize station levels based on flow rates
# station_levels = np.zeros(num_stations, dtype=int)
# for i, station_id in enumerate(station_ids):
#     if station_id in flow_rate_data:
#         flow_rate = flow_rate_data[station_id]
#         if np.isnan(flow_rate):
#             level = 0
#         else:
#             level = (np.round(flow_rate * total_bikes / np.sum(flow_rate)))
#             if np.isnan(level):
#                 level = 0
#             level = np.clip(level, 0, total_capacity)
#         station_levels[i] = level
#     else:
#         station_levels[i] = np.clip(
#             total_capacity // num_stations, min_station_capacity, max_station_capacity)

# # Adjust capacities and levels to meet the constraints
# remaining_bikes = total_bikes - np.sum(station_levels)
# remaining_capacity = total_capacity - np.sum(station_capacities)

# while remaining_bikes > 0 or remaining_capacity > 0:
#     for i in range(num_stations):
#         if remaining_bikes > 0:
#             increment = min(remaining_bikes,
#                             station_capacities[i]-station_levels[i])
#             station_levels[i] += increment
#             remaining_bikes -= increment
#         if remaining_capacity > 0:
#             increment = min(remaining_capacity,
#                             max_station_capacity-station_capacities[i])
#             station_capacities[i] += increment
#             remaining_capacity -= increment
#         if remaining_bikes <= 0 and remaining_capacity <= 0:
#             break

# # Create station instances and store in a dictionary
# stations = {}
# for station_id, capacity, level in zip(station_ids, station_capacities, station_levels):
#     stations[station_id] = Station(
#         station_id=station_id, level=level, capacity=capacity)

# # Print the initial bike list for each station
# for station_id, station in stations.items():
#     print(
#         f"Station {station_id} Initial Bike List: {station.level} / {station.capacity}")

# # Print the sum of levels and capacities for all stations
# print("Level Sum:", sum(station.level for station in stations.values()))
# print("Capacity Sum:", sum(station.capacity for station in stations.values()))


In [24]:


# # Parameters
# num_stations = 20
# total_capacity = 500
# total_bikes = 399
# min_station_capacity = 12
# max_station_capacity = 32

# # Assuming start_flow_df and end_flow_df are available
# start_time = 8
# end_time = 12

# # Calculate flow rate using pandas pivot tables
# start_flow_count = start_flow_df.pivot_table(index='Start ID', columns='Start Time', aggfunc='size', fill_value=0)
# start_flow_count = start_flow_count.loc[:, start_time:end_time]
# end_flow_count = end_flow_df.pivot_table(index='End ID', columns='End Time', aggfunc='size', fill_value=0)
# end_flow_count = end_flow_count.loc[:, start_time:end_time]
# flow_rate = end_flow_count - start_flow_count

# # Calculate flow rate data as a dictionary
# flow_rate_data = flow_rate.sum(axis=1).to_dict()

# # Calculate capacities and levels based on flow rates
# station_ids = list(flow_rate_data.keys())
# flow_rates = np.array(list(flow_rate_data.values()))

# # Normalize flow rates to sum up to the total number of bikes
# normalized_flow_rates = np.round(
#     flow_rates * total_bikes / np.sum(flow_rates)).astype(int)

# # Initialize station capacities and levels
# station_capacities = np.clip(normalized_flow_rates + total_capacity //
#                              num_stations, min_station_capacity, max_station_capacity)
# station_levels = np.clip(normalized_flow_rates, 0, station_capacities)

# # Adjust capacities and levels to meet the constraints
# remaining_bikes = total_bikes - np.sum(station_levels)
# remaining_capacity = total_capacity - np.sum(station_capacities)

# for _ in range(remaining_bikes):
#     idx = np.random.randint(num_stations-1)
#     if station_levels[idx] < station_capacities[idx]:
#         station_levels[idx] += 1

# for _ in range(remaining_capacity):
#     idx = np.random.randint(num_stations-1)
#     station_capacities[idx] += 1

# # Create station instances and store in a dictionary
# stations = {}
# for station_id, capacity, level in zip(station_ids, station_capacities, station_levels):
#     stations[station_id] = Station(station_id=station_id, level=level, capacity=capacity)

# # Print the initial bike list for each station
# for station_id, station in stations.items():
#     print(f"Station {station_id} Initial Bike List: {station.level} / {station.capacity}")

# # Print the sum of levels and capacities for all stations
# print("Level Sum:", sum(station.level for station in stations.values()))
# print("Capacity Sum:", sum(station.capacity for station in stations.values()))


In [25]:
# unique_stations = np.unique(subset_df["Start Station Id"].values)
# num_stations = len(unique_stations)
# total_capacity = 500
# total_bikes = 399
# Stations = {}
# level_sum = []
# capacity_sum = []

# # Calculate the number of bikes to allocate to each station
# bikes_per_station = total_bikes // num_stations

# # Allocate bikes and initialize the stations
# capacity_per_station = total_capacity // num_stations
# for i, station in enumerate(unique_stations):
#     x_i = bikes_per_station // 2
#     r_i = capacity_per_station
#     if i == num_stations - 1:
#         # Allocate the remaining capacity to the last station
#         r_i = total_capacity - capacity_per_station * (num_stations - 1)
#     Stations[station] = Station(station_id=station, level=x_i, capacity=r_i)
#     level_sum.append(x_i)
#     capacity_sum.append(r_i)

# StationDict = {station.id: station for station in Stations.values()}

# # Assign any remaining bikes to the stations with the most available capacity
# remaining_bikes = total_bikes - sum(level_sum)
# while remaining_bikes > 0:
#     # Find the station with the most available capacity
#     max_capacity = -1
#     max_station = None
#     for station_id, station in Stations.items():
#         available_capacity = station.capacity - station.level
#         if available_capacity > max_capacity:
#             max_capacity = available_capacity
#             max_station = station
#     # Assign one bike to the station and update the remaining number of bikes
#     max_station.level += 1
#     remaining_bikes -= 1
#     level_sum[unique_stations.tolist().index(max_station.id)] += 1

# # Print the initial bike list for each station
# level_sum = []
# capacity_sum = []
# for station_id, station in Stations.items():
#     print(f"Station {station_id} Initial Bike List: {station.Get_Bike_List()}")
#     level_sum.append(station.level)
#     capacity_sum.append(station.capacity)

# print("Level Sum", sum(level_sum))
# print("Capacity Sum", sum(capacity_sum))


In [26]:
# # Define the time frame
# start_time = 16
# end_time = 24

# # Calculate the number of stations and bikes per station based on capacity constraints
# unique_stations = np.unique(subset_df["Start Station Id"].values)
# num_stations = len(unique_stations)
# total_capacity = 500
# total_bikes = 399
# avg_capacity = total_capacity // num_stations
# avg_bikes = total_bikes // num_stations

# # Calculate the start and end flow counts for each station
# start_flow_count = start_flow_df.pivot_table(
#     index='Start ID', columns='Start Time', aggfunc='size', fill_value=0)
# start_flow_count = start_flow_count.loc[:, start_time:end_time]
# end_flow_count = end_flow_df.pivot_table(
#     index='End ID', columns='End Time', aggfunc='size', fill_value=0)
# end_flow_count = end_flow_count.loc[:, start_time:end_time]

# # Initialize the stations and create a dictionary mapping station IDs to Station instances
# Stations = {}
# station_capacities = []
# station_levels = []
# for station_id in unique_stations:
#     # Calculate the maximum and minimum number of bikes and docks based on flow rates
#     start_flow = start_flow_count.loc[station_id].values
#     end_flow = end_flow_count.loc[station_id].values
#     net_flow = sum(end_flow) - sum(start_flow)
#     min_bikes = avg_bikes - max(min(start_flow), -net_flow, 0)
#     max_bikes = avg_bikes + max(max(end_flow), -net_flow, 0)
#     # ensure max bikes does not exceed station capacity limit
#     max_bikes = min(max_bikes, 32)
#     max_docks = avg_capacity - max(end_flow)
#     # ensure sum of max_bikes and max_docks does not exceed total capacity
#     max_docks = min(max_docks, total_capacity - max_bikes)

#     # Set the initial level and capacity of the station based on the calculated minimum and maximum values
#     level = min(max(min_bikes, net_flow +max(0, sum(start_flow) - sum(end_flow))), max_bikes)
#     capacity = max_bikes + max_docks
    
#     Stations[station_id] = Station(
#         station_id=station_id, level=level, capacity=capacity)
#     station_capacities.append(capacity)
#     station_levels.append(level)

# # Normalize station capacities and levels to ensure that the total capacity and number of bikes are exactly equal to the target values
# total_station_capacity = sum(station_capacities)
# #capacity_per_station = total_capacity 
# station_capacity_factors = [total_capacity /
#                             total_station_capacity] * num_stations
# for i, station_id in enumerate(unique_stations):
#     Stations[station_id].capacity = int(
#         station_capacities[i] * station_capacity_factors[i])
#     Stations[station_id].level = int(
#         station_levels[i] * total_bikes / sum(station_levels))

# # Print the initial bike list for each station
# level_sum = []
# capacity_sum = []
# for station_id, station in Stations.items():
#     #print(f"Station {station_id} Initial Bike List: {station.Get_Bike_List()}")
#     level_sum.append(station.level)
#     capacity_sum.append(station.capacity)

# print("Level Sum", sum(level_sum))
# print("Capacity Sum", sum(capacity_sum))

# # Allocate any remaining bikes to the stations with the most available capacity
# # Assign any remaining bikes to the stations while considering the flow rate
# remaining_bikes = total_bikes - sum(level_sum)
# while remaining_bikes > 0:
#     # Find the station with the most available capacity
#     max_capacity = 32
#     max_station = None
#     for station_id, station in Stations.items():
#         available_capacity = station.capacity - station.level
#         if available_capacity > max_capacity:
#             max_capacity = available_capacity
#             max_station = station
#     # Check the flow rate of the station
#     start_flow = start_flow_count.loc[max_station.id].values
#     end_flow = end_flow_count.loc[max_station.id].values
#     net_flow = sum(end_flow) - sum(start_flow)
#     # If there is positive net flow, remove a bike
#     if net_flow > 0:
#         max_station.level -= 1
#         remaining_bikes -= 1
#     # If there is negative net flow, add a bike
#     elif net_flow < 0:
#         if max_station.level < max_station.capacity:
#             max_station.level += 1
#             remaining_bikes -= 1
#     # If there is no net flow, add or remove a bike based on the available capacity
#     else:
#         if max_station.level < max_station.capacity:
#             max_station.level += 1
#             remaining_bikes -= 1
#         else:
#             max_station.level -= 1
#             remaining_bikes -= 1

# # Create a new dictionary to store the updated station instances
# # Create a new dictionary to store the updated station instances
# updated_stations = {}
# final_stations = {}
# capacity_sum = 0
# # Iterate over the original Stations dictionary
# # Iterate over the original Stations dictionary
# for i, (station_id, station) in enumerate(Stations.items()):
#     # Update the station instance and store it in the new dictionary
#     updated_station = Station(station_id=station_id,
#                               level=station.level, capacity=station.capacity)
#     updated_stations[station_id] = updated_station

#     # Allocate capacity to the stations based on the average capacity and the remaining capacity
#     r_i = avg_capacity
#     if i == num_stations - 1:
#         r_i = total_capacity - avg_capacity * (num_stations - 1)
#     r_i = min(r_i, 32)  # enforce maximum capacity constraint
#     updated_station.capacity = r_i

#     print(f"Station {station_id} Initial Bike List: {updated_station.Get_Bike_List()}")
#     capacity_sum += updated_station.capacity
#     final_stations[station_id] = Station(
#         station_id=station_id, level=updated_station.level, capacity=r_i)


# # Replace the original Stations dictionary with the final one
# Stations = final_stations

# # Print the sum of levels and capacities for all stations
# level_sum = [station.level for station in Stations.values()]
# capacity_sum = [station.capacity for station in Stations.values()]
# print("Level Sum:", sum(level_sum))
# print("Capacity Sum:", sum(capacity_sum))
# print("Total Capacity:", sum(capacity_sum))


In [27]:
# # Define the time frame
# start_time = 16
# end_time = 24

# # Calculate the number of stations and bikes per station based on capacity constraints
# unique_stations = np.unique(subset_df["Start Station Id"].values)
# num_stations = len(unique_stations)
# total_capacity = 500
# total_bikes = 399
# avg_capacity = total_capacity // num_stations
# avg_bikes = total_bikes // num_stations

# # Calculate the start and end flow counts for each station
# start_flow_count = start_flow_df.pivot_table(
#     index='Start ID', columns='Start Time', aggfunc='size', fill_value=0)
# start_flow_count = start_flow_count.loc[:, start_time:end_time]
# end_flow_count = end_flow_df.pivot_table(
#     index='End ID', columns='End Time', aggfunc='size', fill_value=0)
# end_flow_count = end_flow_count.loc[:, start_time:end_time]

# # Initialize the stations and create a dictionary mapping station IDs to Station instances
# Stations = {}
# station_capacities = []
# station_levels = []
# for station_id in unique_stations:
#     # Calculate the maximum and minimum number of bikes and docks based on flow rates
#     start_flow = start_flow_count.loc[station_id].values
#     end_flow = end_flow_count.loc[station_id].values
#     net_flow = sum(end_flow) - sum(start_flow)
#     min_bikes = avg_bikes - max(min(start_flow), -net_flow, 0)
#     max_bikes = avg_bikes + max(max(end_flow), -net_flow, 0)
#     # ensure max bikes does not exceed station capacity limit
#     max_bikes = min(max_bikes, 32)
#     max_docks = avg_capacity - max(end_flow)
#     # ensure sum of max_bikes and max_docks does not exceed total capacity
#     max_docks = min(max_docks, total_capacity - max_bikes)

#     # Set the initial level and capacity of the station based on the calculated minimum and maximum values
#     level = min(max(min_bikes, net_flow +
#                 max(0, sum(start_flow) - sum(end_flow))), max_bikes)
#     capacity = max_bikes + max_docks
#     capacity = min(capacity, 32)  # enforce maximum capacity constraint

#     Stations[station_id] = Station(
#         station_id=station_id, level=level, capacity=capacity)
#     station_capacities.append(capacity)
#     station_levels.append(level)

# # Normalize station capacities and levels to ensure that the total capacity and number of bikes are exactly equal to the target values
# total_station_capacity = sum(station_capacities)
# #capacity_per_station = total_capacity
# station_capacity_factors = [total_capacity /
#                             total_station_capacity] * num_stations
# for i, station_id in enumerate(unique_stations):
#     Stations[station_id].capacity = int(
#         station_capacities[i] * station_capacity_factors[i])
#     Stations[station_id].level = int(
#         station_levels[i] * total_bikes / sum(station_levels))

# # Print the initial bike list for each station
# level_sum = []
# capacity_sum = []
# for station_id, station in Stations.items():
#     #print(f"Station {station_id} Initial Bike List: {station.Get_Bike_List()}")
#     level_sum.append(station.level)
#     capacity_sum.append(station.capacity)

# print("Level Sum", sum(level_sum))
# print("Capacity Sum", sum(capacity_sum))

# # Allocate any remaining bikes to the stations with the most available capacity
# # Assign any remaining bikes to the stations while considering the flow rate
# remaining_bikes = total_bikes - sum(level_sum)
# while remaining_bikes > 0:
#     # Find the station with the most available capacity
#     max_capacity = -1
#     max_station = None
#     for station_id, station in Stations.items():
#         available_capacity = station.capacity - station.level
#         if available_capacity > max_capacity:
#             max_capacity = available_capacity
#             max_station = station
#     # Check the flow rate of the station
#     start_flow = start_flow_count.loc[max_station.id].values
#     end_flow = end_flow_count.loc[max_station.id].values
#     net_flow = sum(end_flow) - sum(start_flow)
#     # If there is positive net flow, remove a bike
#     if net_flow > 0:
#         max_station.level -= 1
#         remaining_bikes -= 1
#     # If there is negative net flow, add a bike
#     elif net_flow < 0:
#         if max_station.level < max_station.capacity:
#             max_station.level += 1
#             remaining_bikes -= 1
#     # If there is no net flow, add or remove a bike based on the available capacity
#     else:
#         if max_station.level < max_station.capacity:
#             max_station.level += 1
#             remaining_bikes -= 1
#         else:
#             max_station.level -= 1
#             remaining_bikes -= 1

# # Create a new dictionary to store the updated station instances
# # Create a new dictionary to store the updated station instances
# updated_stations = {}
# final_stations = {}
# capacity_sum = 0
# # Iterate over the original Stations dictionary
# # Iterate over the original Stations dictionary
# for i, (station_id, station) in enumerate(Stations.items()):
#     # Update the station instance and store it in the new dictionary
#     updated_station = Station(station_id=station_id,
#                               level=station.level, capacity=station.capacity)
#     updated_stations[station_id] = updated_station

#     # Allocate capacity to the stations based on the average capacity and the remaining capacity
#     r_i = avg_capacity
#     if i == num_stations - 1:
#         r_i = total_capacity - avg_capacity * (num_stations - 1)
#     r_i = min(r_i, 32)  # enforce maximum capacity constraint
#     updated_station.capacity = r_i

#     print(
#         f"Station {station_id} Initial Bike List: {updated_station.Get_Bike_List()}")
#     capacity_sum += updated_station.capacity
#     final_stations[station_id] = Station(station_id=station_id, level=updated_station.level, capacity=r_i)


# # Replace the original Stations dictionary with the final one
# Stations = final_stations

# # Print the sum of levels and capacities for all stations
# level_sum = [station.level for station in Stations.values()]
# capacity_sum = [station.capacity for station in Stations.values()]
# print("Level Sum:", sum(level_sum))
# print("Capacity Sum:", sum(capacity_sum))
# print("Total Capacity:", sum(capacity_sum))


# Optimizing Bike Allocations

- Time Frame: 8am - 12pm
- number of stations = 20
- station capcities: 12 <= r_i <= 32
- Bike per station: 0 <= x_i <= r_i
- total capacity of stations = 500
- total number of bikes = 399


# RUN SIMULATION FUNCTION

In [36]:
def RUN_SIMULATION(StationDict, T, Calendar):
    global CI_total_error_list, CI_Full_Error_list, CI_Empty_Error_list
    global CI_inter_arrival_time_list, CI_arrival_time_list, CI_start_station_id_list, CI_end_station_id_list, CI_start_time_list, CI_end_time_list, CI_trip_time_list
    
    total_error_list = []
    for _ in range(3):
        NextCustomerID.counter = 0

        ZSimRNG = SimRNG_Modified.InitializeRNSeed()

        TheCTStats = []
        TheDTStats = []
        TheQueues = []
        TheResources = []

        total_capacity = 500
        total_bikes = 399
        Full_Error = 0
        Empty_Error = 0
        inital_count = 0

        total_capacity = 500
        total_bikes = 399
        Stations = {}
        level_sum = 0
        level_sum = 0
        capacity_sum = 0
        Full_Error = 0
        Empty_Error = 0
        inital_count = 0

        Full_Error = 0
        Empty_Error = 0
        inital_count = 0

        ############################################################################################################################################################################
        unique_stations = np.unique(subset_df["Start Station Id"].values)
        num_stations = len(unique_stations)
        total_capacity = 500
        total_bikes = 399
        Stations = {}
        level_sum = 0
        level_sum = 0
        capacity_sum = 0
        Full_Error = 0
        Empty_Error = 0
        inital_count = 0
        objective_fun = 0

        ##################################

    ############################################################################################################################################################################
        val_level_sum = []
        val_capacity_sum = []
        for station_id, station in StationDict.items():
            #print(f"Station {station_id} Initial Bike List: {station.Get_Bike_List()}")
            val_level_sum.append(station.level)
            val_capacity_sum.append(station.capacity)
        print("Level Sum", sum(val_level_sum))
        print("Capacity Sum", sum(val_capacity_sum))
    ############################################################################################################################################################################

        SimFunctions.SimFunctionsInit(
            Calendar, TheQueues, TheCTStats, TheDTStats, TheResources)
        SimFunctions.Schedule(Calendar, "Start", 0)

        NextEvent = Calendar.Remove()
        SimClasses.Clock = NextEvent.EventTime
        if NextEvent.EventType == "Start":
            Start()

    ############################################################################################################################################################################
    # SIMULATION RUN
        for T in range(15, 25):  # T = hours intervals
            inital_count += 1
            hour = T // 2
            minute = 00 if T % 2 == 0 else 30
            unit = 'PM' if hour >= 12 else 'AM'
            if T == 16 or T == 20 or T == 24:
                print("Interval:", T)
            mini = 0
            SimFunctions.Schedule(Calendar, "inital_Customer_Arrival", 0)

            if inital_count == 2:
                Full_Error = 0
                Empty_Error = 0

            while True:
                #print("Clock: {:02d}:{:02d} {:s}".format(hour, minute, unit))
                NextEvent = Calendar.Remove()
                SimClasses.Clock = NextEvent.EventTime
                minute = int((SimClasses.Clock) % 60)
                if SimClasses.Clock >= (T+1) * 30:
                    break
                if NextEvent.EventType == "inital_Customer_Arrival":
                    Empty_Error = inital_Customer_Arrival(
                        Empty_Error, CustomerList, T, minute)
                elif NextEvent.EventType == "Customer_Arrival":
                    Empty_Error = Customer_Arrival(
                        Empty_Error, CustomerList, T, minute)
                elif NextEvent.EventType == "Bike_Arrival":
                    Full_Error = Bike_Arrival(Full_Error, CustomerList, T, minute)

            if T == 24:
                while Calendar.N() > 0:
                    NextEvent = Calendar.Remove()
                    SimClasses.Clock = NextEvent.EventTime
                    minute = int((SimClasses.Clock) % 60)
                    if NextEvent.EventType == "Bike_Arrival":
                        Full_Error = Bike_Arrival(
                            Full_Error, CustomerList, T, minute)
    ##################################################################################################################################################

        objective_fun = Full_Error + Empty_Error
        total_error_list.append(objective_fun)
        
        
        CI_total_error_list.append(objective_fun)
        CI_Full_Error_list.append(Full_Error)
        CI_Empty_Error_list.append(Empty_Error)

        CI_inter_arrival_time_list.append(inter_arrival_time_list)
        CI_arrival_time_list.append(arrival_time_list)
        CI_start_station_id_list.append(start_station_id_list)
        CI_end_station_id_list.append(end_station_id_list)
        CI_start_time_list.append(start_time_list)
        CI_end_time_list.append(end_time_list)
        CI_trip_time_list.append(trip_time_list)
        
    ci_95_error = CI_95(total_error_list)
    var_error = np.var(total_error_list)
    print("CI 95 of Expected Errors", ci_95_error)
    print("Variance of Error", var_error)
    #print(f"End of BikeSim")
    print("--------------------------------------------------------------------------------------------------------------------------------------------------")
    
    return CI_95(total_error_list)[0]


# Heuristic 1 Optimization

In [37]:
def generate_trial_solution(Stations, w):
    statE = [station_id for station_id, station in Stations.items() if station.level + w <= station.capacity]
    statF = [station_id for station_id,station in Stations.items() if station.level - w >= 0]
    
    np.random.seed(np.random.randint(1, 1000000))
    
    sE = np.random.choice(statE)
    sF = np.random.choice(statF)

    new_stations = deepcopy(Stations)
    new_stations[sE].level += w
    new_stations[sF].level -= w

    return new_stations

In [39]:
inter_arrival_time_list = []
arrival_time_list = []
start_station_id_list = []
end_station_id_list = []
start_time_list = []
end_time_list = []
trip_time_list = []

Stations = []
CustomerList = []
CI_Full_Error_list = []
CI_Empty_Error_list = []

CI_inter_arrival_time_list = []
CI_arrival_time_list = []
CI_start_station_id_list = []
CI_end_station_id_list = []
CI_start_time_list = []
CI_end_time_list = []
CI_trip_time_list = []
CI_total_error_list = []

Full_Error = 0
Empty_Error = 0
inital_count = 0

Stations = []
CustomerList = []

Calendar = SimClasses.EventCalendar()
ZSimRNG = SimRNG_Modified.InitializeRNSeed()

unique_stations = np.unique(subset_df["Start Station Id"].values)
num_stations = len(unique_stations)
total_capacity = 500
total_bikes = 399
Stations = {}
level_sum = 0
level_sum = 0
capacity_sum = 0
Full_Error = 0
Empty_Error = 0
inital_count = 0
w = 4

############################################################################################################################################################################
# Initialize the stations and create a dictionary mapping station IDs to Station instances
for i, station in enumerate(unique_stations):
    x_i = total_bikes // 19
    r_i = capacity_per_station
    if i == num_stations - 1:
        # Allocate the remaining capacity to the last station
        r_i = total_capacity - capacity_per_station * (num_stations - 1)
    Stations[station] = Station(station_id=station, level=x_i, capacity=r_i)
    level_sum += x_i
    for station_id, station in Stations.items():
        capacity_sum += station.capacity

StationDict = {station.id: station for station in Stations.values()}

Obj_Fun = RUN_SIMULATION(StationDict, T, Calendar)

Errors_DF = pd.DataFrame({"Full Error": [CI_Full_Error_list[-1]],
                          "Empty Error": [CI_Empty_Error_list[-1]],
                          "Total Error": [CI_total_error_list[-1]]}, index=[0])

BikeSim_DF = pd.DataFrame({"Start Station ID": CI_start_station_id_list,
                           "End Station ID": CI_end_station_id_list,
                           "Arrival Rate": CI_arrival_time_list,
                           "Interarrival Rate": CI_inter_arrival_time_list,
                           "Start Time": CI_start_time_list,
                           "End Time": CI_end_time_list,
                           "Trip Time": CI_trip_time_list})

station_id = BikeSim_DF["Start Station ID"].values[0]
end_station_id = BikeSim_DF["End Station ID"].values[0]
start_times = BikeSim_DF["Start Time"].values[0]
end_times = BikeSim_DF["End Time"].values[0]
trip_times = BikeSim_DF["Trip Time"].values[0]
Arrival_Rates = BikeSim_DF["Arrival Rate"].values[0]
Interarrival_Rates = BikeSim_DF["Interarrival Rate"].values[0]

start_flow_df = pd.DataFrame({"Start ID": station_id, "Start Time": start_times})
end_flow_df = pd.DataFrame({"End ID": end_station_id, "End Time": end_times, "Trip Time": trip_times})
arrival_flow_df = pd.DataFrame({"Start ID": station_id, "Arrival Rates": Arrival_Rates})
interarrival_flow_df = pd.DataFrame({"Start ID": station_id, "Arrival Rates": Interarrival_Rates})

start_flow_count = start_flow_df.pivot_table(index='Start ID', columns='Start Time', aggfunc='size', fill_value=0)
start_flow_count = start_flow_count.loc[:, 16:]
end_flow_count = end_flow_df.pivot_table(index='End ID', columns='End Time', aggfunc='size', fill_value=0)
end_flow_count = end_flow_count.loc[:, 16:]
flow_rate = end_flow_count - start_flow_count

#INITAL TRIAL
############################################################################################################################################################################

Stations = {}
total_bikes_assigned = 0
max_capacity = 32
total_bikes = 399

for i, (station_id, station) in enumerate(StationDict.items()):
    x_i = station.level
    r_i = station.capacity
    Stations[station_id] = Station(station_id=station_id, level=x_i, capacity=r_i)
    total_bikes_assigned += x_i

remaining_bikes = total_bikes - total_bikes_assigned
sorted_stations = sorted(Stations.items(), key=lambda x: x[1].level)
stations_copy = deepcopy(Stations)

while remaining_bikes > 0:
    for station_id, station in sorted_stations:
        if stations_copy[station_id].level < max_capacity and remaining_bikes > 0:
            stations_copy[station_id].level += 1
            remaining_bikes -= 1
            if remaining_bikes == 0:
                break

OPT_STATIONS = {station.id: station for station in stations_copy.values()}
############################################################################################################################################################################
#OPTIMIZE

best_total_error = float('inf')
best_bike_list = {}

num_iterations = 2
num_replications = 3


for reps in range(num_iterations):
    Calendar = SimClasses.EventCalendar()
    ZSimRNG = SimRNG_Modified.InitializeRNSeed()
    # Generate trial solution
    trial_solution = generate_trial_solution(OPT_STATIONS, w)
    trial_solution = {station.id: station for station in trial_solution.values()}

    # Simulate and evaluate
    new_total_error = 0
    for _ in range(num_replications):
        Calendar = SimClasses.EventCalendar()
        ZSimRNG = SimRNG_Modified.InitializeRNSeed()            
        total_error_rep = RUN_SIMULATION(trial_solution, T, Calendar)
        new_total_error += total_error_rep

    new_total_error /= num_replications

    if new_total_error < best_total_error:
        best_total_error = new_total_error
        OPT_STATIONS = trial_solution

        statE = [station_id for station_id, station in OPT_STATIONS.items()
                 if station.level < station.capacity]
        statF = [station_id for station_id,
                 station in OPT_STATIONS.items() if station.level > 0]

print("Best Bike List")
best_bike_list = {station.id: station for station in OPT_STATIONS.values()}
for station_id, station in OPT_STATIONS.items():
    print(f"Station {station_id} Initial Bike List: {station.Get_Bike_List()}")




Level Sum 399
Capacity Sum 500
Interval: 16
Interval: 20
Interval: 24
Level Sum 339
Capacity Sum 500
Interval: 16
Interval: 20
Interval: 24
Level Sum 314
Capacity Sum 500
Interval: 16
Choosing Route Error NO DATA found for start station: '7217' and start T: '19'
Choosing Route Error NO DATA found for start station: '7217' and start T: '19'
Choosing Route Error NO DATA found for start station: '7217' and start T: '19'
Interval: 20
Interval: 24
CI 95 of Expected Errors (669.0, '+/-', 109.78333085370171)
Variance of Error 6274.666666666667
--------------------------------------------------------------------------------------------------------------------------------------------------
Level Sum 399
Capacity Sum 500
Interval: 16
Choosing Route Error NO DATA found for start station: '7217' and start T: '19'
Choosing Route Error NO DATA found for start station: '7217' and start T: '19'
Choosing Route Error NO DATA found for start station: '7217' and start T: '19'
Choosing Route Error NO DATA 

In [ ]:

# for reps in range(num_iterations):

#     Calendar = SimClasses.EventCalendar()
#     ZSimRNG = SimRNG_Modified.InitializeRNSeed()
#     OPT_STATIONS = {}
#     total_bikes_assigned = 0
#     max_capacity = 32
#     total_bikes = 399

#     for i, (station_id, station) in enumerate(StationDict.items()):
#         x_i = station.level
#         r_i = station.capacity
#         OPT_STATIONS[station_id] = Station(
#             station_id=station_id, level=x_i, capacity=r_i)
#         total_bikes_assigned += x_i

#     remaining_bikes = total_bikes - total_bikes_assigned
#     sorted_stations = sorted(OPT_STATIONS.items(), key=lambda x: x[1].level)
#     stations_copy = deepcopy(OPT_STATIONS)

#     while remaining_bikes > 0:
#         for station_id, station in sorted_stations:
#             if stations_copy[station_id].level < max_capacity and remaining_bikes > 0:
#                 stations_copy[station_id].level += 1
#                 remaining_bikes -= 1
#                 if remaining_bikes == 0:
#                     break

#     StationDict = {station.id: station for station in stations_copy.values()}
#     for station_id, station in StationDict.items():
#         print(
#             f"Station {station_id} Initial Bike List: {station.Get_Bike_List()}")

#     # Calculate the occupancy for each station
#     occupancy = {station_id: station.level /
#                  station.capacity for station_id, station in OPT_STATIONS.items()}

#     # Find the stations with the highest and lowest occupancy
#     station_idF = max(occupancy, key=occupancy.get)
#     station_idE = min(occupancy, key=occupancy.get)

#     # Move w bikes between the two stations
#     if OPT_STATIONS[station_idF].level - w >= 0 and OPT_STATIONS[station_idE].level + w <= OPT_STATIONS[station_idE].capacity:
#         OPT_STATIONS[station_idF].level += w
#         OPT_STATIONS[station_idE].level -= w
#         print("Moved bikes from Station", station_idF)
#         print("To Station", station_idE)

#     print("New List")
#     for station_id, station in OPT_STATIONS.items():
#         print(
#             f"Station {station_id} Initial Bike List: {station.Get_Bike_List()}")

#     StationDict = {station.id: station for station in OPT_STATIONS.values()}

#     new_total_error = RUN_SIMULATION(StationDict, T, Calendar)

#     # If the new total error is lower than the best total error, update the best bike list
#     if new_total_error < best_total_error:
#         best_total_error = new_total_error
#         best_bike_list = {station.id: station.Get_Bike_List()
#                           for station in OPT_STATIONS.values()}


In [ ]:
# # CI_95(CI_arrival_time_list[0])

# K = 9
# alpha = 0.05
# x = (1-alpha)**(1/(K-1))
# n_reps = 100
# df = n-1
# ti = t.ppf(x, df)
# th = ti

# s = [20, 20, 20, 20, 40, 40, 40, 60, 60]
# S = [40, 60, 80, 100, 60, 80, 100, 80, 100]

# w_ih = 0
# w_ih_list = []

# I = set()

# for i in range(len(s)):
#     for j in range(len(S)):
#         if i != j:
#             w_ih = np.sqrt(
#                 (ti**2) * (sample_var_poly[i] / n_reps) + (th**2) * (sample_var_poly[j] / n_reps))
#             if sample_means_poly[i] > sample_means_poly[j] + w_ih:
#                 break
#             else:
#                 I.add((s[i], S[i]))


# Extra Optimization

### Optimize Based on Flow Rate

In [ ]:
# num_iterations = 5

# inter_arrival_time_list = []
# arrival_time_list = []
# start_station_id_list = []
# end_station_id_list = []
# start_time_list = []
# end_time_list = []
# trip_time_list = []

# Stations = []
# CustomerList = []
# CI_Full_Error_list = []
# CI_Empty_Error_list = []

# CI_inter_arrival_time_list = []
# CI_arrival_time_list = []
# CI_start_station_id_list = []
# CI_end_station_id_list = []
# CI_start_time_list = []
# CI_end_time_list = []
# CI_trip_time_list = []
# CI_total_error_list = []

# Full_Error = 0
# Empty_Error = 0
# inital_count = 0

# Stations = []
# CustomerList = []

# Calendar = SimClasses.EventCalendar()
# ZSimRNG = SimRNG_Modified.InitializeRNSeed()

# unique_stations = np.unique(subset_df["Start Station Id"].values)
# num_stations = len(unique_stations)
# total_capacity = 500
# total_bikes = 399
# Stations = {}
# level_sum = 0
# level_sum = 0
# capacity_sum = 0
# Full_Error = 0
# Empty_Error = 0
# inital_count = 0
# w = 4

# best_total_error = float('inf')
# best_bike_list = {}

# ############################################################################################################################################################################
# # Initialize the stations and create a dictionary mapping station IDs to Station instances
# for i, station in enumerate(unique_stations):
#     x_i = total_bikes // 19
#     r_i = capacity_per_station
#     if i == num_stations - 1:
#         # Allocate the remaining capacity to the last station
#         r_i = total_capacity - capacity_per_station * (num_stations - 1)
#     Stations[station] = Station(station_id=station, level=x_i, capacity=r_i)
#     level_sum += x_i
#     for station_id, station in Stations.items():
#         capacity_sum += station.capacity

# StationDict = {station.id: station for station in Stations.values()}

# Obj_Fun = RUN_SIMULATION(StationDict, T, Calendar)

# Errors_DF = pd.DataFrame({"Full Error": [CI_Full_Error_list[-1]],
#                           "Empty Error": [CI_Empty_Error_list[-1]],
#                           "Total Error": [CI_total_error_list[-1]]}, index=[0])

# BikeSim_DF = pd.DataFrame({"Start Station ID": CI_start_station_id_list,
#                            "End Station ID": CI_end_station_id_list,
#                            "Arrival Rate": CI_arrival_time_list,
#                            "Interarrival Rate": CI_inter_arrival_time_list,
#                            "Start Time": CI_start_time_list,
#                            "End Time": CI_end_time_list,
#                            "Trip Time": CI_trip_time_list})

# station_id = BikeSim_DF["Start Station ID"].values[0]
# end_station_id = BikeSim_DF["End Station ID"].values[0]
# start_times = BikeSim_DF["Start Time"].values[0]
# end_times = BikeSim_DF["End Time"].values[0]
# trip_times = BikeSim_DF["Trip Time"].values[0]
# Arrival_Rates = BikeSim_DF["Arrival Rate"].values[0]
# Interarrival_Rates = BikeSim_DF["Interarrival Rate"].values[0]

# start_flow_df = pd.DataFrame({"Start ID": station_id, "Start Time": start_times})
# end_flow_df = pd.DataFrame({"End ID": end_station_id, "End Time": end_times, "Trip Time": trip_times})
# arrival_flow_df = pd.DataFrame({"Start ID": station_id, "Arrival Rates": Arrival_Rates})
# interarrival_flow_df = pd.DataFrame({"Start ID": station_id, "Arrival Rates": Interarrival_Rates})

# start_flow_count = start_flow_df.pivot_table(index='Start ID', columns='Start Time', aggfunc='size', fill_value=0)
# start_flow_count = start_flow_count.loc[:, 16:]
# end_flow_count = end_flow_df.pivot_table(index='End ID', columns='End Time', aggfunc='size', fill_value=0)
# end_flow_count = end_flow_count.loc[:, 16:]
# flow_rate = end_flow_count - start_flow_count

# ############################################################################################################################################################################
# for reps in range(num_iterations):

#     Calendar = SimClasses.EventCalendar()
#     ZSimRNG = SimRNG_Modified.InitializeRNSeed()
#     Stations = {}
#     total_bikes_assigned = 0
#     max_capacity = 32
#     total_bikes = 399
    
#     for i, (station_id, station) in enumerate(StationDict.items()):
#         x_i = station.level
#         r_i = station.capacity
#         Stations[station_id] = Station(station_id=station_id, level=x_i, capacity=r_i)
#         total_bikes_assigned += x_i

#     remaining_bikes = total_bikes - total_bikes_assigned
#     sorted_stations = sorted(Stations.items(), key=lambda x: x[1].level)
#     stations_copy = deepcopy(Stations)

#     while remaining_bikes > 0:
#         for station_id, station in sorted_stations:
#             if stations_copy[station_id].level < max_capacity and remaining_bikes > 0:
#                 stations_copy[station_id].level += 1
#                 remaining_bikes -= 1
#                 if remaining_bikes == 0:
#                     break

#     StationDict = {station.id: station for station in stations_copy.values()}
#     for station_id, station in StationDict.items():
#         print(f"Station {station_id} Initial Bike List: {station.Get_Bike_List()}")

#     break
#     # Find the stations with the highest and lowest flow rates
#     station_id1 = flow_rate.sum(axis=1).idxmax()
#     station_id2 = flow_rate.sum(axis=1).idxmin()

#     for station_id, station in Stations.items():
#         print(f"Station {station_id} Initial Bike List: {station.Get_Bike_List()}")

#     # Move w bikes between the two stations
#     Stations[station_id1].level -= w
#     Stations[station_id2].level += w

#     # Check if the new bike list is valid (no negative bike levels)
#     if Stations[station_id1].level < 0:
#         continue

#     # Update the StationDict with the new bike list
#     StationDict = {station.id: station for station in Stations.values()}

#     # Run your simulation with the updated StationDict and calculate the new total error
#     new_total_error = RUN_SIMULATION(StationDict, T, Calendar)

#     # If the new total error is lower than the best total error, update the best bike list
#     if new_total_error < best_total_error:
#         best_total_error = new_total_error
#         best_bike_list = {station.id: station.Get_Bike_List() for station in Stations.values()}


In [ ]:
# # CI_95(CI_arrival_time_list[0])

# K = 9
# alpha = 0.05
# x = (1-alpha)**(1/(K-1))
# n_reps = 100
# df = n-1
# ti = t.ppf(x, df)
# th = ti

# s = [20, 20, 20, 20, 40, 40, 40, 60, 60]
# S = [40, 60, 80, 100, 60, 80, 100, 80, 100]

# w_ih = 0
# w_ih_list = []

# I = set()

# for i in range(len(s)):
#     for j in range(len(S)):
#         if i != j:
#             w_ih = np.sqrt(
#                 (ti**2) * (sample_var_poly[i] / n_reps) + (th**2) * (sample_var_poly[j] / n_reps))
#             if sample_means_poly[i] > sample_means_poly[j] + w_ih:
#                 break
#             else:
#                 I.add((s[i], S[i]))


### Individual Heuristic

In [43]:
# Initialize the stations and create a dictionary mapping station IDs to Station instances
for i, station in enumerate(unique_stations):
    x_i = total_bikes // 19
    r_i = capacity_per_station
    if i == num_stations - 1:
        # Allocate the remaining capacity to the last station
        r_i = total_capacity - capacity_per_station * (num_stations - 1)
    Stations[station] = Station(station_id=station, level=x_i, capacity=r_i)
    level_sum += x_i
    for station_id, station in Stations.items():
        capacity_sum += station.capacity

StationDict = {station.id: station for station in Stations.values()}

Obj_Fun = RUN_SIMULATION(StationDict, T, Calendar)


Level Sum 399
Capacity Sum 500


KeyError: 0

In [40]:
num_iterations = 5

inter_arrival_time_list = []
arrival_time_list = []
start_station_id_list = []
end_station_id_list = []
start_time_list = []
end_time_list = []
trip_time_list = []

Stations = []
CustomerList = []
CI_Full_Error_list = []
CI_Empty_Error_list = []

CI_inter_arrival_time_list = []
CI_arrival_time_list = []
CI_start_station_id_list = []
CI_end_station_id_list = []
CI_start_time_list = []
CI_end_time_list = []
CI_trip_time_list = []
CI_total_error_list = []

Full_Error = 0
Empty_Error = 0
inital_count = 0

Stations = []
CustomerList = []

Calendar = SimClasses.EventCalendar()
ZSimRNG = SimRNG_Modified.InitializeRNSeed()

unique_stations = np.unique(subset_df["Start Station Id"].values)
num_stations = len(unique_stations)
total_capacity = 500
total_bikes = 399
Stations = {}
level_sum = 0
level_sum = 0
capacity_sum = 0
Full_Error = 0
Empty_Error = 0
inital_count = 0
w = 4

best_total_error = float('inf')
best_bike_list = {}

############################################################################################################################################################################
# Initialize the stations and create a dictionary mapping station IDs to Station instances
for i, station in enumerate(unique_stations):
    x_i = total_bikes // 19
    r_i = capacity_per_station
    if i == num_stations - 1:
        # Allocate the remaining capacity to the last station
        r_i = total_capacity - capacity_per_station * (num_stations - 1)
    Stations[station] = Station(station_id=station, level=x_i, capacity=r_i)
    level_sum += x_i
    for station_id, station in Stations.items():
        capacity_sum += station.capacity

StationDict = {station.id: station for station in Stations.values()}

Obj_Fun = RUN_SIMULATION(StationDict, T, Calendar)

Errors_DF = pd.DataFrame({"Full Error": [CI_Full_Error_list[-1]],
                          "Empty Error": [CI_Empty_Error_list[-1]],
                          "Total Error": [CI_total_error_list[-1]]}, index=[0])

BikeSim_DF = pd.DataFrame({"Start Station ID": CI_start_station_id_list,
                           "End Station ID": CI_end_station_id_list,
                           "Arrival Rate": CI_arrival_time_list,
                           "Interarrival Rate": CI_inter_arrival_time_list,
                           "Start Time": CI_start_time_list,
                           "End Time": CI_end_time_list,
                           "Trip Time": CI_trip_time_list})

station_id = BikeSim_DF["Start Station ID"].values[0]
end_station_id = BikeSim_DF["End Station ID"].values[0]
start_times = BikeSim_DF["Start Time"].values[0]
end_times = BikeSim_DF["End Time"].values[0]
trip_times = BikeSim_DF["Trip Time"].values[0]
Arrival_Rates = BikeSim_DF["Arrival Rate"].values[0]
Interarrival_Rates = BikeSim_DF["Interarrival Rate"].values[0]

start_flow_df = pd.DataFrame(
    {"Start ID": station_id, "Start Time": start_times})
end_flow_df = pd.DataFrame(
    {"End ID": end_station_id, "End Time": end_times, "Trip Time": trip_times})
arrival_flow_df = pd.DataFrame(
    {"Start ID": station_id, "Arrival Rates": Arrival_Rates})
interarrival_flow_df = pd.DataFrame(
    {"Start ID": station_id, "Arrival Rates": Interarrival_Rates})

start_flow_count = start_flow_df.pivot_table(
    index='Start ID', columns='Start Time', aggfunc='size', fill_value=0)
start_flow_count = start_flow_count.loc[:, 16:]
end_flow_count = end_flow_df.pivot_table(
    index='End ID', columns='End Time', aggfunc='size', fill_value=0)
end_flow_count = end_flow_count.loc[:, 16:]
flow_rate = end_flow_count - start_flow_count

############################################################################################################################################################################
for reps in range(num_iterations):

    Calendar = SimClasses.EventCalendar()
    ZSimRNG = SimRNG_Modified.InitializeRNSeed()
    Stations = {}
    total_bikes_assigned = 0
    max_capacity = 32
    total_bikes = 399

    for i, (station_id, station) in enumerate(StationDict.items()):
        x_i = station.level
        r_i = station.capacity
        Stations[station_id] = Station(station_id=station_id, level=x_i, capacity=r_i)
        total_bikes_assigned += x_i

    remaining_bikes = total_bikes - total_bikes_assigned
    sorted_stations = sorted(Stations.items(), key=lambda x: x[1].level)
    stations_copy = deepcopy(Stations)

    while remaining_bikes > 0:
        for station_id, station in sorted_stations:
            if stations_copy[station_id].level < max_capacity and remaining_bikes > 0:
                stations_copy[station_id].level += 1
                remaining_bikes -= 1
                if remaining_bikes == 0:
                    break

    # StationDict = {station.id: station for station in stations_copy.values()}
    # for station_id, station in StationDict.items():
    #     print(f"Station {station_id} Initial Bike List: {station.Get_Bike_List()}")

    # Calculate the occupancy for each station
    occupancy = {station_id: station.level / station.capacity for station_id, station in Stations.items()}

    # Find the stations with the highest and lowest occupancy
    station_idF = max(occupancy, key=occupancy.get)
    station_idE = min(occupancy, key=occupancy.get)

    # Move w bikes between the two stations
    if Stations[station_idF].level - w >= 0 and Stations[station_idE].level + w <= Stations[station_idE].capacity:
        Stations[station_idF].level += w
        Stations[station_idE].level -= w
        print("Moved bikes from Station", station_idF)
        print("To Station", station_idE)

    print("New List")
    for station_id, station in Stations.items():
        print(f"Station {station_id} Initial Bike List: {station.Get_Bike_List()}")

    StationDict = {station.id: station for station in Stations.values()}

    new_total_error = RUN_SIMULATION(StationDict, T, Calendar)

    # If the new total error is lower than the best total error, update the best bike list
    if new_total_error < best_total_error:
        best_total_error = new_total_error
        best_bike_list = {station.id: station.Get_Bike_List()
                          for station in Stations.values()}


Level Sum 399
Capacity Sum 500
Interval: 16
Interval: 20
Interval: 24
Level Sum 345
Capacity Sum 500
Interval: 16
Choosing Route Error NO DATA found for start station: '7217' and start T: '19'
Choosing Route Error NO DATA found for start station: '7217' and start T: '19'
Choosing Route Error NO DATA found for start station: '7217' and start T: '19'
Choosing Route Error NO DATA found for start station: '7217' and start T: '19'
Choosing Route Error NO DATA found for start station: '7217' and start T: '19'
Choosing Route Error NO DATA found for start station: '7217' and start T: '19'
Choosing Route Error NO DATA found for start station: '7217' and start T: '19'
Interval: 20
Interval: 24
Level Sum 316
Capacity Sum 500
Interval: 16
Choosing Route Error NO DATA found for start station: '7217' and start T: '19'
Interval: 20
Interval: 24
CI 95 of Expected Errors (731.3333333333334, '+/-', 105.04047812364738)
Variance of Error 5744.222222222223
--------------------------------------------------

KeyError: 0

In [ ]:
# from random import choice


# def generate_trial_solution(Stations, w):
#     statE = [station_id for station_id, station in Stations.items() if station.level + w <= station.capacity]
#     statF = [station_id for station_id,station in Stations.items() if station.level - w >= 0]

#     sE = choice(statE)
#     sF = choice(statF)

#     new_stations = deepcopy(Stations)
#     new_stations[sE].level += w
#     new_stations[sF].level -= w

#     return new_stations


# iterations = 3
# num_replications = 10

# for _ in range(iterations):
#     trial_solution = generate_trial_solution(Stations, w)

#     new_total_error = 0
#     for _ in range(num_replications):
#         #ZimRNG = ZSimRNG.SeedInitialize()
#         total_error_rep = RUN_SIMULATION(trial_solution, T, Calendar)
#         new_total_error += total_error_rep

#     new_total_error /= num_replications

#     if new_total_error < best_total_error:
#         best_total_error = new_total_error
#         Stations = trial_solution

#         statE = [station_id for station_id, station in Stations.items()
#                  if station.level < station.capacity]
#         statF = [station_id for station_id,
#                  station in Stations.items() if station.level > 0]

# print("Best Bike List")
# for station_id, station in Stations.items():
#     print(f"Station {station_id} Initial Bike List: {station.Get_Bike_List()}")


In [ ]:
# for reps in range(num_iterations):

#     # Generate trial solution
#     trial_solution = generate_trial_solution(Stations, w)

#     # Simulate and evaluate
#     new_total_error = 0
#     for _ in range(num_replications):
#         ZSimRNG.SeedInitialize(seed)
#         seed += 1
#         total_error_rep = RUN_SIMULATION(trial_solution, T, Calendar)
#         new_total_error += total_error_rep

#     new_total_error /= num_replications

#     if new_total_error < best_total_error:
#         best_total_error = new_total_error
#         Stations = trial_solution

#         statE = [station_id for station_id, station in Stations.items()
#                  if station.level < station.capacity]
#         statF = [station_id for station_id,
#                  station in Stations.items() if station.level > 0]

# print("Best Bike List")
# for station_id, station in Stations.items():
#     print(f"Station {station_id} Initial Bike List: {station.Get_Bike_List()}")


# Scrap

In [ ]:
def RUN_SIMULATION(StationDict, T, Calendar):
    global CI_total_error_list, CI_Full_Error_list, CI_Empty_Error_list
    global CI_inter_arrival_time_list, CI_arrival_time_list, CI_start_station_id_list, CI_end_station_id_list, CI_start_time_list, CI_end_time_list, CI_trip_time_list
    NextCustomerID.counter = 0

    ZSimRNG = SimRNG_Modified.InitializeRNSeed()

    TheCTStats = []
    TheDTStats = []
    TheQueues = []
    TheResources = []

    total_capacity = 500
    total_bikes = 399
    Full_Error = 0
    Empty_Error = 0
    inital_count = 0

    total_capacity = 500
    total_bikes = 399
    Stations = {}
    level_sum = 0
    level_sum = 0
    capacity_sum = 0
    Full_Error = 0
    Empty_Error = 0
    inital_count = 0

    Full_Error = 0
    Empty_Error = 0
    inital_count = 0

    ############################################################################################################################################################################
    unique_stations = np.unique(subset_df["Start Station Id"].values)
    num_stations = len(unique_stations)
    total_capacity = 500
    total_bikes = 399
    Stations = {}
    level_sum = 0
    level_sum = 0
    capacity_sum = 0
    Full_Error = 0
    Empty_Error = 0
    inital_count = 0
    objective_fun = 0

    ##################################

############################################################################################################################################################################
    val_level_sum = []
    val_capacity_sum = []
    for station_id, station in StationDict.items():
        print(
            f"Station {station_id} Initial Bike List: {station.Get_Bike_List()}")
        val_level_sum.append(station.level)
        val_capacity_sum.append(station.capacity)
    print("Level Sum", sum(val_level_sum))
    print("Capacity Sum", sum(val_capacity_sum))
############################################################################################################################################################################

    SimFunctions.SimFunctionsInit(
        Calendar, TheQueues, TheCTStats, TheDTStats, TheResources)
    SimFunctions.Schedule(Calendar, "Start", 0)

    NextEvent = Calendar.Remove()
    SimClasses.Clock = NextEvent.EventTime
    if NextEvent.EventType == "Start":
        Start()

############################################################################################################################################################################
# SIMULATION RUN
    for T in range(15, 25):  # T = hours intervals
        inital_count += 1
        hour = T // 2
        minute = 00 if T % 2 == 0 else 30
        unit = 'PM' if hour >= 12 else 'AM'
        if T == 16 or T == 20 or T == 24:
            print("Interval:", T)
        mini = 0
        SimFunctions.Schedule(Calendar, "inital_Customer_Arrival", 0)

        if inital_count == 2:
            Full_Error = 0
            Empty_Error = 0

        while True:
            #print("Clock: {:02d}:{:02d} {:s}".format(hour, minute, unit))
            NextEvent = Calendar.Remove()
            SimClasses.Clock = NextEvent.EventTime
            minute = int((SimClasses.Clock) % 60)
            if SimClasses.Clock >= (T+1) * 30:
                break
            if NextEvent.EventType == "inital_Customer_Arrival":
                Empty_Error = inital_Customer_Arrival(
                    Empty_Error, CustomerList, T, minute)
            elif NextEvent.EventType == "Customer_Arrival":
                Empty_Error = Customer_Arrival(
                    Empty_Error, CustomerList, T, minute)
            elif NextEvent.EventType == "Bike_Arrival":
                Full_Error = Bike_Arrival(Full_Error, CustomerList, T, minute)

        if T == 24:
            while Calendar.N() > 0:
                NextEvent = Calendar.Remove()
                SimClasses.Clock = NextEvent.EventTime
                minute = int((SimClasses.Clock) % 60)
                if NextEvent.EventType == "Bike_Arrival":
                    Full_Error = Bike_Arrival(
                        Full_Error, CustomerList, T, minute)
##################################################################################################################################################

    objective_fun = Full_Error + Empty_Error
    CI_total_error_list.append(objective_fun)
    CI_Full_Error_list.append(Full_Error)
    CI_Empty_Error_list.append(Empty_Error)

    CI_inter_arrival_time_list.append(inter_arrival_time_list)
    CI_arrival_time_list.append(arrival_time_list)
    CI_start_station_id_list.append(start_station_id_list)
    CI_end_station_id_list.append(end_station_id_list)
    CI_start_time_list.append(start_time_list)
    CI_end_time_list.append(end_time_list)
    CI_trip_time_list.append(trip_time_list)

    print("ERRORS", objective_fun)
    print(f"End of BikeSim")
    print("--------------------------------------------------------------------------------------------------------------------------------------------------")
    return objective_fun


In [ ]:
inter_arrival_time_list = []
arrival_time_list = []
start_station_id_list = []
end_station_id_list = []
start_time_list = []
end_time_list = []
trip_time_list = []

Stations = []
CustomerList = []
CI_Full_Error_list = []
CI_Empty_Error_list = []

CI_inter_arrival_time_list = []
CI_arrival_time_list = []
CI_start_station_id_list = []
CI_end_station_id_list = []
CI_start_time_list = []
CI_end_time_list = []
CI_trip_time_list = []
CI_total_error_list = []
new_total_error_list = []

Full_Error = 0
Empty_Error = 0
inital_count = 0

Stations = []
CustomerList = []

Calendar = SimClasses.EventCalendar()
ZSimRNG = SimRNG_Modified.InitializeRNSeed()

unique_stations = np.unique(subset_df["Start Station Id"].values)
num_stations = len(unique_stations)
total_capacity = 500
total_bikes = 399
Stations = {}
level_sum = 0
level_sum = 0
capacity_sum = 0
Full_Error = 0
Empty_Error = 0
inital_count = 0
w = 4

############################################################################################################################################################################
# Initialize the stations and create a dictionary mapping station IDs to Station instances
for i, station in enumerate(unique_stations):
    x_i = total_bikes // 19
    r_i = capacity_per_station
    if i == num_stations - 1:
        # Allocate the remaining capacity to the last station
        r_i = total_capacity - capacity_per_station * (num_stations - 1)
    Stations[station] = Station(station_id=station, level=x_i, capacity=r_i)
    level_sum += x_i
    for station_id, station in Stations.items():
        capacity_sum += station.capacity

StationDict = {station.id: station for station in Stations.values()}

Obj_Fun = RUN_SIMULATION(StationDict, T, Calendar)

Errors_DF = pd.DataFrame({"Full Error": [CI_Full_Error_list[-1]],
                          "Empty Error": [CI_Empty_Error_list[-1]],
                          "Total Error": [CI_total_error_list[-1]]}, index=[0])

BikeSim_DF = pd.DataFrame({"Start Station ID": CI_start_station_id_list,
                           "End Station ID": CI_end_station_id_list,
                           "Arrival Rate": CI_arrival_time_list,
                           "Interarrival Rate": CI_inter_arrival_time_list,
                           "Start Time": CI_start_time_list,
                           "End Time": CI_end_time_list,
                           "Trip Time": CI_trip_time_list})

station_id = BikeSim_DF["Start Station ID"].values[0]
end_station_id = BikeSim_DF["End Station ID"].values[0]
start_times = BikeSim_DF["Start Time"].values[0]
end_times = BikeSim_DF["End Time"].values[0]
trip_times = BikeSim_DF["Trip Time"].values[0]
Arrival_Rates = BikeSim_DF["Arrival Rate"].values[0]
Interarrival_Rates = BikeSim_DF["Interarrival Rate"].values[0]

start_flow_df = pd.DataFrame(
    {"Start ID": station_id, "Start Time": start_times})
end_flow_df = pd.DataFrame(
    {"End ID": end_station_id, "End Time": end_times, "Trip Time": trip_times})
arrival_flow_df = pd.DataFrame(
    {"Start ID": station_id, "Arrival Rates": Arrival_Rates})
interarrival_flow_df = pd.DataFrame(
    {"Start ID": station_id, "Arrival Rates": Interarrival_Rates})

start_flow_count = start_flow_df.pivot_table(
    index='Start ID', columns='Start Time', aggfunc='size', fill_value=0)
start_flow_count = start_flow_count.loc[:, 16:]
end_flow_count = end_flow_df.pivot_table(
    index='End ID', columns='End Time', aggfunc='size', fill_value=0)
end_flow_count = end_flow_count.loc[:, 16:]
flow_rate = end_flow_count - start_flow_count

# # INITAL TRIAL
# ############################################################################################################################################################################


############################################################################################################################################################################
# OPTIMIZE

best_total_error = float('inf')
best_bike_list = {}

num_iterations = 15
num_replications = 3


for reps in range(num_iterations):
    Calendar = SimClasses.EventCalendar()
    ZSimRNG = SimRNG_Modified.InitializeRNSeed()
    Stations = {}
    total_bikes_assigned = 0
    max_capacity = 32
    total_bikes = 399

    for i, (station_id, station) in enumerate(StationDict.items()):
        x_i = station.level
        r_i = station.capacity
        Stations[station_id] = Station(station_id=station_id, level=x_i, capacity=r_i)
        total_bikes_assigned += x_i

    remaining_bikes = total_bikes - total_bikes_assigned
    sorted_stations = sorted(Stations.items(), key=lambda x: x[1].level)
    stations_copy = deepcopy(Stations)

    while remaining_bikes > 0:
        for station_id, station in sorted_stations:
            if stations_copy[station_id].level < max_capacity and remaining_bikes > 0:
                stations_copy[station_id].level += 1
                remaining_bikes -= 1
                if remaining_bikes == 0:
                    break
                
    # Find the stations with the highest and lowest flow rates
    station_id1 = flow_rate.sum(axis=1).idxmax()
    station_id2 = flow_rate.sum(axis=1).idxmin()

    # Move w bikes between the two stations
    Stations[station_id1].level -= w
    Stations[station_id2].level += w

    # Check if the new bike list is valid (no negative bike levels)
    if Stations[station_id1].level < 0:
        continue

    # Update the StationDict with the new bike list
    StationDict = {station.id: station for station in Stations.values()}
    # Run your simulation with the updated bike list and calculate the new total error
    new_total_error = run_simulation(Stations, T, Calendar)
    new_total_error_list.append(new_total_error)
    # If the new total error is lower than the best total error, update the best bike list
    if new_total_error < best_total_error:
        best_total_error = new_total_error
        best_bike_list = StationDict.copy()


In [ ]:
plt.plot(new_total_error_list)
